In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('..')

In [4]:
from hdna import * 
import nupack as nu
import networkx as nx 

In [44]:
model = Model('dna', '3D', stacking='nostacking')
s1 = Strand(model, "ACCAAACCAC")
s2 = s1.complementary()
C = Chamber(model, s1, s2)
N = Kinetwork(model, s1, s2, Geometry(120,270))
S = Simulator(model, N)

In [9]:
N.overview

{None: 0,
 'singlestranded': 1,
 'duplex': 1,
 'zipping': 44,
 'on_nucleation': 10,
 'off_nucleation': 26,
 'backfray': 24,
 'sliding': 8}

In [38]:
FG = N.filternodes('state', lambda x: x == 'sliding', N.DG)
list(FG.nodes.data())

[('((........+))........',
  {'obj': <hdna.complex.Complex at 0x7ff705974d60>,
   'pairs': 2,
   'state': 'sliding',
   'dpxdist': 8,
   'dtc': 0,
   'side': 'l'}),
 ('........((+........))',
  {'obj': <hdna.complex.Complex at 0x7ff705974b50>,
   'pairs': 2,
   'state': 'sliding',
   'dpxdist': 8,
   'dtc': 0,
   'side': 'r'}),
 ('((((......+.)))).....',
  {'obj': <hdna.complex.Complex at 0x7ff7059a4820>,
   'pairs': 4,
   'state': 'sliding',
   'dpxdist': 5,
   'dtc': 0,
   'side': 'l'}),
 ('.....((((.+......))))',
  {'obj': <hdna.complex.Complex at 0x7ff7059745e0>,
   'pairs': 4,
   'state': 'sliding',
   'dpxdist': 5,
   'dtc': 0,
   'side': 'r'}),
 ('(....((...+))....)...',
  {'obj': <hdna.complex.Complex at 0x7ff7059a77f0>,
   'pairs': 3,
   'state': 'sliding',
   'dpxdist': 3,
   'dtc': 0,
   'side': 'l'}),
 ('...(....((+...))....)',
  {'obj': <hdna.complex.Complex at 0x7ff7059a4610>,
   'pairs': 3,
   'state': 'sliding',
   'dpxdist': 3,
   'dtc': 0,
   'side': 'r'}),
 ('.(.((.(

In [41]:
def ownsliding(node, graph):
        """ special filtering method to get the goal sliding associated with an off_nucleation"""
        dpxdist = node[1]['dpxdist']
        side    = node[1]['side']
        def filternode(node):
            try: return graph.nodes[node]['dpxdist'] == dpxdist and graph.nodes[node]['side'] == side and graph.nodes[node]['state'] == 'sliding'
            except KeyError: pass     
        sliding = list(nx.subgraph_view(graph, filter_node=filternode).nodes())[0]
        return sliding

In [28]:
off = list(FG.nodes.data())[3]
print(*off, sep='\n')

.........(+........).
{'obj': <hdna.complex.Complex object at 0x7ff705974970>, 'pairs': 1, 'state': 'off_nucleation', 'dpxdist': 8, 'dtc': 1, 'side': 'r'}


In [42]:
ownsliding(off, N.DG)

'........((+........))'

In [ ]:
for n in N.DG.nodes.data():
    try: n[1]['obj'] = str(type(n[1]['obj']))   
    except: pass 
nx.write_gexf(N.DG, './COMPLETE.gexf')